Mypy is an optional static type checker for Python that aims to combine the benefits of dynamic (or "duck") typing and static typing. Mypy combines the expressive power and convenience of Python with a powerful type system and compile-time type checking. Mypy type checks standard Python programs; run them using any Python VM with basically no runtime overhead.

`nb_mypy` Pozwala na sprawdzanie typów wewnątrz notebooków

In [27]:
%load_ext nb_mypy

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


## Proste typy

In [34]:
v1: int = 1
v2: float = 1.23
v3: str = "hello"
v4: bool = True
v5: None = None

v6: int | str = 123

## Struktury danych

In [45]:
# from typing import List, Set, Dict, Tuple

v7: list[int] = [1, 2, 3]
v8: set[str] = {"a", "b", "C"}
v9: dict[str, int] = {"a": 1, "b": 2}
v10: tuple[int, str, bool] = (1, "abc", False)

# v7: List[int] = [1, 2, 3]
# v8: Set[str] = {"a", "b", "C"}
# v9: Dict[str, int] = {"a": 1, "b": 2}
# v10: Tuple[int, str, bool] = (1, "abc", False)

## Aliasy i nowe typy

In [59]:
from typing import NewType

Vector = list[float]
UserId = NewType('UserId', int)

In [61]:
from functools import reduce

def dot(a: Vector, b: Vector) -> float:
  return reduce(lambda x, y: x + y, map(lambda x: x[0] * x[1], zip(a, b)))

# listy są inwariantne
vector1: list[float] = [1, 2]

print(dot(vector1, [3, 4]))

11


In [77]:
def save_user(user: UserId):
  print("user", user, "saved!")

save_user(UserId(1))
# save_user(2)

# user3: UserId = UserId(1) + UserId(2)
user3: int = UserId(1) + UserId(2)

user 1 saved!


## Callable

In [76]:
from typing import Callable

def add(a: int, b: int) -> int:
  return a + b

def kind_of_zip(a: list[int], b: list[int], op: Callable[[int, int], int]) -> list[int]:
  return list(map(lambda t: op(*t), zip(a, b)))

print(kind_of_zip([1, 2, 3], [4, 5, 6], add))

[5, 7, 9]


## Tuple (i FrozenSet) są kowariantne

T jest kowariantny, jezeli:

`A <: B => T[A] <: T[B]`

In [101]:
class Animal:
  def eat(self):
    print('Animal eats')

class Dog(Animal):
  def bark(self):
    print('Dog barks')

  def eat(self):
    print('Dog eats')

class Cat(Animal):
  def meow(self):
    print('Cat meows')

  def eat(self):
    print('Cat eats')

In [102]:
tuple_of_dogs = (Dog(), Dog(), Dog())

def sum(x: tuple[Animal, Animal, Animal]):
  for animal in x:
    animal.eat()

sum(tuple_of_dogs)

Dog eats
Dog eats
Dog eats


## Callable jest kowariantny ze względu na zwracany typ

In [103]:
from typing import Callable

def make_dog() -> Dog:
  return Dog()

def add_animal(lst: list[Animal], op: Callable[[], Animal]):
  lst.append(op())

animals: list[Animal] = [Cat(), Cat()]
print(animals)

add_animal(animals, make_dog)

print(animals)

[<__main__.Cat object at 0x17f38c850>, <__main__.Cat object at 0x14f87e890>]
[<__main__.Cat object at 0x17f38c850>, <__main__.Cat object at 0x14f87e890>, <__main__.Dog object at 0x15d89e350>]


## Callable jest kontrawariantny ze względu na typy argumentów

T jest kontrawariantny, jezeli:

`A <: B => T[A] :> T[B]`

In [106]:
from typing import Callable

def feed_animal(animal: Animal):
  animal.eat()

def feed_cats(lst: list[Cat], feed: Callable[[Cat], None]):
  for cat in lst:
    cat.eat()

cats: list[Cat] = [Cat(), Cat()]

feed_cats(cats, feed_animal)


Cat eats
Cat eats


## Mutowalne kolekcje są inwariantne

T jest inwariantny, jezeli:

`A <: B => ~(T[A] <: T[B]) && ~(T[A] :> T[B])`

In [ ]:
# czemu nie są kontrawariantne jest w miarę oczywiste, ale brak kowariancji moze taki nie być

# list_of_dogs = [Dog(), Dog()]
# list_of_animals = list_of_dogs
# list_of_animals.append(Cat())

# for dog in list_of_dogs:
#   dog.bark()

## Generyki

In [113]:
from typing import TypeVar, Sequence

T = TypeVar('T')

def first(seq: Sequence[T]) -> T | None:
  return seq[0] if len(seq) > 0 else None

print(first([]))
print(first([1, 2, 3]))

None
1


In [129]:
from typing import TypeVar, Generic

T = TypeVar('T')

class Optional(Generic[T]):
  def __init__(self, value=None):
    self.__value = value
  
  def get(self) -> T | None:
    return self.__value

  def set(self, value: T):
    self.__value = value

  def clear(self):
    self.__value = None
  
  def empty(self):
    return self.__value == None

optional_intlist: Optional[list[int]] = Optional()
print(optional_intlist.empty())

optional_string_set: Optional[set[str]] = Optional({"a", "b"})
print(optional_string_set.empty())

True
False


## Any

In [139]:
from typing import Any

def better_print(*args: Any):
  print(' '.join(map(str, args)).upper())
  
better_print(123, "hello", "world", Cat())

123 HELLO WORLD <__MAIN__.CAT OBJECT AT 0X289741DD0>


## Protokoły

In [146]:
from typing import Sized

# Sized -> __len__
# Hashable -> __hash__
# Container -> __contains__
# Callable -> __call__
# Iterable -> __iter__
# Iterator -> __iter__, __next__
# Reversible -> __reversed__
# ... https://docs.python.org/3/library/collections.abc.html, większość ma aliasy w typing

class LenableObj(Sized):
  def __init__(self, size: int):
    self.__size = size

  def __len__(self):
    return self.__size

print(len(LenableObj(5)))

5


## Structural subtyping

In [156]:
from typing import Sized

def fake_len(obj: Sized):
  pass

class NonLenableObj():
  def __init__(self, size: int):
    self.__size = size

class LenableObjNoInheritance():
  def __init__(self, size: int):
    self.__size = size

  def __len__(self):
    return self.__size

fake_len(NonLenableObj(5))
fake_len(LenableObjNoInheritance(5))

<cell>17: error: Argument 1 to "fake_len" has incompatible type "NonLenableObj"; expected "Sized"  [arg-type]


## Method overloading

In [154]:
from typing import overload

class Processor:
  @overload
  def process(self, response: int):
    pass

  # def test(self):
  #   pass

  @overload
  def process(self, response: str):
    pass

  def process(self, response):
    print("Received response:", response)

Processor().process("Hello world")
Processor().process(42)
Processor().process(Cat())

<cell>4: error: Single overload definition, multiple required  [misc]
<cell>11: error: Name "process" already defined on line 4  [no-redef]
<cell>11: error: Single overload definition, multiple required  [misc]
<cell>18: error: Argument 1 to "process" of "Processor" has incompatible type "str"; expected "int"  [arg-type]
<cell>20: error: Argument 1 to "process" of "Processor" has incompatible type "Cat"; expected "int"  [arg-type]


Received response: Hello world
Received response: 42
Received response: <__main__.Cat object at 0x16a0b63d0>
